In [1]:
import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [66]:
# Define the language model to use.
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [83]:
# Define the tools to use.
import json
import datetime
from collections import defaultdict
from langchain.agents import tool
from typing import Dict, Text, Any, List
from pydantic.v1 import BaseModel, Field


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def get_peloton_classes() -> Dict[Text, Any]:
    """Get recent Peloton classes."""
    response = json.load(open("peloton_classes.json", "r"))

    today = datetime.datetime.today().date()
    recent_classes = {}
    for w in response['data']:
        workout_date = datetime.datetime.fromtimestamp(w['original_air_time']).date()

        if (today - workout_date).days > 7:
            break
        
        recent_classes[w['id']] = w

    return recent_classes


@tool
def get_recent_user_workouts() -> Dict[Text, Any]:
    """Get the user's Peloton workouts from the past week."""
    response = json.load(open("user_workouts.json", "r"))

    today = datetime.datetime.today().date()
    recent_workouts = defaultdict(list)
    for w in response['data']:
        workout_date = datetime.datetime.fromtimestamp(w['created_at']).date()

        # Only get workouts from the last 7 days.
        if (today - workout_date).days > 7:
            break

        if 'ride' in w:
            title = w['ride']['title']
        elif 'peloton' in w:
            title = w['peloton']['ride']['title']
        else:
            title = "Unknown"
            
        lbl = f"{workout_date}: {title}"

        recent_workouts[str(workout_date)].append(lbl)

    return recent_workouts


class StackInput(BaseModel):
    recommended_workout: List[Dict[Text, Any]] = Field(description="the list of recommended classes.")


@tool(args_schema=StackInput)
def add_class_to_stack(recommended_workout: List[Dict[Text, Any]]):
    """Allows a user to add selected workout to the Peloton stack if the user explicitly asks to."""
    print(recommended_workout)


# @tool
# def recommend_workout(user_classes: Dict[Text, Any], peloton_classes: Dict[Text, Any]) -> Dict[Text, Any]:
#     """Given the user's recent classes and available Peloton classes recommend a workout for the user."""
    


tools = [get_word_length, get_recent_user_workouts, get_peloton_classes, add_class_to_stack]

In [84]:
# Tell the LLM about the tools available via `bind`.
from langchain.tools.render import format_tool_to_openai_function

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [85]:
# Define the agent.
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.messages import AIMessage, HumanMessage

chat_history = []

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [86]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [87]:
input1 = "Recommend a workout for me."
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)



> Entering new AgentExecutor chain...

Invoking: `get_recent_user_workouts` with `{}`


defaultdict(<class 'list'>, {'2023-12-02': ['2023-12-02: 10 min Holiday Full Body Stretch', '2023-12-02: 20 min Pop Ride'], '2023-12-01': ['2023-12-01: 10 min Relaxing Meditation'], '2023-11-30': ['2023-11-30: 10 min Full Body Stretch', '2023-11-30: 20 min Strength Roll Call: Full Body'], '2023-11-29': ['2023-11-29: 10 min Full Body Stretch', '2023-11-29: 20 min Low Impact Ride'], '2023-11-27': ['2023-11-27: 10 min Full Body Stretch', '2023-11-27: 20 min Recovery Ride'], '2023-11-26': ['2023-11-26: 15 min Low Impact Ride', '2023-11-26: 20 min Upper Body Strength']})
Invoking: `get_peloton_classes` with `{}`


{'f5bf0720d7bc43a5ae0da0b17cfda975': {'free_for_limited_time': False, 'content_availability': 'available', 'content_availability_level': 'digital_and_above', 'is_limited_ride': False, 'availability': {'is_available': True, 'reason': None}, 'class_type_ids': ['d84d91191eb54d4aa265f82fe3375b92'

In [88]:
agent_executor.invoke({"input": "Yes", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `add_class_to_stack` with `{}`




ValidationError: 1 validation error for StackInput
recommended_workout
  field required (type=value_error.missing)